# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install ipython-sql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db

/home/christopher/code/datasci223_assignment3/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

In [2]:
%%sql
SELECT name
FROM sqlite_master
where type = 'table'

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [5]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [6]:
%%sql
INSERT INTO solution VALUES (1, 'Insert the name of the person you found here');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
That's not the right person. Try again!
